In [1]:
import numpy as np
import tensorflow as tf
import util
import numpy as np
import tensorflow as tf
import util
import torch
from keras.utils.np_utils import *
from keras.datasets import mnist
MNIST_CLASSIFIER_FROZEN_GRAPH ='/home/imi432_006/huangdengrong/GAN-Research/metrics_gan/metrics/classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def calculate_labels(images,epoch,global_step):
    label_dict={}
    for i in range(10):
        label_dict[i]=0
    eval_images = tf.convert_to_tensor(images)
    y_logit = util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    labels=tf.argmax(y_logit, 1)
    labels=tf.Session().run(labels)
    for data in labels:
        label_dict[data]+=1
    for i in range(10):
        label_dict[i]=label_dict[i]/len(images)
    max_value=max(label_dict.values())
    min_value=min(label_dict.values())
    print('epoch:%d   global_step:%d'%(epoch,global_step))
    print(label_dict)
    print('chazhi:%.8f'%(max_value-min_value))
    return label_dict,max_value-min_value

Using TensorFlow backend.


In [3]:
from __future__ import print_function, division

from keras.datasets import fashion_mnist,mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 7 * 7, input_dim=self.latent_dim))
        model.add(Reshape((7,7, 512)))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3,strides=1,padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, strides=2,kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(self.img_rows*self.img_cols*self.channels, input_dim=np.prod(self.img_shape)))
        model.add(Reshape((self.img_rows,self.img_cols,self.channels)))

        model.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps=[]
        values=[]

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
#                 print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    label_dict,value=self.mode_drop(epoch,global_step)
                    steps.append(global_step)
                    values.append(value)
#             plt.subplots(1, 1)
        plt.plot(steps,values)
        plt.xlim([0,40000])
        plt.ylim([0.0,1.0])
        plt.xlabel('steps')
        plt.ylabel('epochs')
        plt.tick_params(axis='both',which='major',labelsize=13)
        plt.legend(loc='lower right')
        if not os.path.isdir('images_cgan'):
            os.mkdir('images_cgan')
        plt.savefig("images_cgan/mode_drop.png")
        plt.show()


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_cgan'):
            os.mkdir('images_cgan')
        fig.savefig("images_cgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()
    def mode_drop(self,epoch,global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        gen_imgs = 0.5 * gen_imgs + 0.5
        label_dict,value=calculate_labels(gen_imgs,epoch,global_step)
        return label_dict,value


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=30, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.gfile.GFile.
epoch:0   global_step:200
{0: 0.0013, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0002, 5: 0.994, 6: 0.0022, 7: 0.0, 8: 0.0017, 9: 0.0006}
chazhi:0.99400000
epoch:0   global_step:400
{0: 0.001, 1: 0.0009, 2: 0.0009, 3: 0.0007, 4: 0.0007, 5: 0.0008, 6: 0.001, 7: 0.9917, 8: 0.0012, 9: 0.0011}
chazhi:0.99100000
epoch:0   global_step:600
{0: 0.0011, 1: 0.001, 2: 0.001, 3: 0.0008, 4: 0.001, 5: 0.0008, 6: 0.0013, 7: 0.9915, 8: 0.001, 9: 0.0005}
chazhi:0.99100000
epoch:0   global_step:800
{0: 0.0012, 1: 0.001, 2: 0.0008, 3: 0.001, 4: 0.0012, 5: 0.0006, 6: 0.0012, 7: 0.991, 8: 0.001, 9: 0.001}
chazhi:0.99040000
epoch:1   global_step:1000
{0: 0.0011, 1: 0.0009, 2: 0.0009, 3: 0.0015, 4: 0.0009, 5: 0.0007, 6: 0.0009, 7: 0.9916, 8: 0.0009, 9: 0.0006}
chazhi:0.99100000
epoch:1   global_step:1200
{0: 0.001, 1: 0.0011, 2: 0.0008, 3: 0.0009, 4: 0.0012, 5: 0.0011, 6: 0.001, 7: 0.991, 8: 0.0009, 9: 0.001}
chazhi:0.99020000
epoch:1   global_step:1400
{0: 0.001, 1: 0.001, 2

epoch:11   global_step:10600
{0: 0.001, 1: 0.001, 2: 0.0009, 3: 0.001, 4: 0.0009, 5: 0.001, 6: 0.0011, 7: 0.9912, 8: 0.001, 9: 0.0009}
chazhi:0.99030000
epoch:11   global_step:10800
{0: 0.001, 1: 0.001, 2: 0.001, 3: 0.0011, 4: 0.001, 5: 0.0008, 6: 0.0011, 7: 0.991, 8: 0.0011, 9: 0.0009}
chazhi:0.99020000
epoch:11   global_step:11000
{0: 0.001, 1: 0.001, 2: 0.0009, 3: 0.0012, 4: 0.0012, 5: 0.0007, 6: 0.0011, 7: 0.991, 8: 0.001, 9: 0.0009}
chazhi:0.99030000
epoch:11   global_step:11200
{0: 0.001, 1: 0.0009, 2: 0.0007, 3: 0.0009, 4: 0.001, 5: 0.001, 6: 0.001, 7: 0.9912, 8: 0.0013, 9: 0.001}
chazhi:0.99050000
epoch:12   global_step:11400
{0: 0.001, 1: 0.001, 2: 0.0008, 3: 0.0009, 4: 0.0011, 5: 0.001, 6: 0.001, 7: 0.9911, 8: 0.001, 9: 0.0011}
chazhi:0.99030000
epoch:12   global_step:11600
{0: 0.0009, 1: 0.001, 2: 0.001, 3: 0.0009, 4: 0.0011, 5: 0.001, 6: 0.0011, 7: 0.991, 8: 0.0011, 9: 0.0009}
chazhi:0.99010000
epoch:12   global_step:11800
{0: 0.0011, 1: 0.001, 2: 0.001, 3: 0.001, 4: 0.001,

epoch:22   global_step:21400
{0: 0.001, 1: 0.001, 2: 0.001, 3: 0.001, 4: 0.001, 5: 0.0009, 6: 0.001, 7: 0.9912, 8: 0.001, 9: 0.0009}
chazhi:0.99030000
epoch:23   global_step:21600
{0: 0.0011, 1: 0.001, 2: 0.0011, 3: 0.0009, 4: 0.0011, 5: 0.0005, 6: 0.001, 7: 0.991, 8: 0.0013, 9: 0.001}
chazhi:0.99050000
epoch:23   global_step:21800
{0: 0.001, 1: 0.001, 2: 0.001, 3: 0.001, 4: 0.0011, 5: 0.0009, 6: 0.001, 7: 0.9911, 8: 0.001, 9: 0.0009}
chazhi:0.99020000
epoch:23   global_step:22000
{0: 0.001, 1: 0.001, 2: 0.0009, 3: 0.0009, 4: 0.0012, 5: 0.0009, 6: 0.0011, 7: 0.9911, 8: 0.0011, 9: 0.0008}
chazhi:0.99030000
epoch:23   global_step:22200
{0: 0.001, 1: 0.001, 2: 0.001, 3: 0.001, 4: 0.001, 5: 0.001, 6: 0.001, 7: 0.991, 8: 0.0011, 9: 0.0009}
chazhi:0.99010000
epoch:23   global_step:22400
{0: 0.0011, 1: 0.001, 2: 0.0008, 3: 0.001, 4: 0.0011, 5: 0.0009, 6: 0.0011, 7: 0.991, 8: 0.001, 9: 0.001}
chazhi:0.99020000
epoch:24   global_step:22600
{0: 0.001, 1: 0.001, 2: 0.001, 3: 0.001, 4: 0.001, 5: 0

No handles with labels found to put in legend.


<Figure size 640x480 with 1 Axes>